#Project_Draft_G_Model_Syntax 

This notebook explores the approach of using a classification model on syntactic parse trees to identify actions.


##Set up Drive

In [0]:
#authorize Colab to access Drive
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


##Set up spaCy

code adapted from https://heartbeat.fritz.ai/nlp-chronicles-intro-to-spacy-34949f1bc118#08c8

spaCy documentation  https://spacy.io/usage

spaCy POS tags
https://spacy.io/api/annotation#pos-tagging

In [0]:
pip install -U spaCy

     |████████████████████████████████| 10.4MB 2.8MB/s 
     |████████████████████████████████| 3.7MB 21.9MB/s 
     |████████████████████████████████| 2.2MB 19.7MB/s 
     |████████████████████████████████| 122kB 40.8MB/s 
  Found existing installation: blis 0.2.4
    Uninstalling blis-0.2.4:
      Successfully uninstalled blis-0.2.4
  Found existing installation: preshed 2.0.1
    Uninstalling preshed-2.0.1:
      Successfully uninstalled preshed-2.0.1
  Found existing installation: thinc 7.0.8
    Uninstalling thinc-7.0.8:
      Successfully uninstalled thinc-7.0.8
  Found existing installation: spacy 2.1.9
    Uninstalling spacy-2.1.9:
      Successfully uninstalled spacy-2.1.9


In [0]:
!python -m spacy download en

     |████████████████████████████████| 12.0MB 4.0MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-cp36-none-any.whl size=12011741 sha256=4cdf2de40276b19b2487feee631de30637070aa9ae7017baee418b6d437b0ed5
  Stored in directory: /tmp/pip-ephem-wheel-cache-mwd2vrrm/wheels/6a/47/fb/6b5a0b8906d8e8779246c67d4658fd8a544d4a03a75520197a
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.1.0
    Uninstalling en-core-web-sm-2.1.0:
      Successfully uninstalled en-core-web-sm-2.1.0
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [0]:
import spacy
from spacy import displacy
nlp=spacy.load('en')

## Corpus

A list of 820 positive examples of 'actions that a citizen could do' was provided by the business in a csv file.

It was necessary to obtain negative examples as well. This was done by extracting random sentences from the corpus into a .txt file, and then manually filtering and editing them in Notepad to make sure they were **not** 'actions that a citizen could do'. This provided 388 non-action sentences.

#### Reserved Test Data 

In [0]:
# manually construct a list of eligible files that have not been examined yet, 
# and may contain both actions and non-actions that have never been seen before
test_files = [
  'coastal_flooded_land_guidelines.txt',
  'En56-226-2008-eng.txt',
  'env-yukon-state-play-analysis-climate-change-impacts-adaptation.txt',
  'FBC_WaterGuide_FINAL.txt',
  'final_climate_change_and_health_backgrounder_overview.txt',
  'Guidebook-2016.txt',
  'HP5-122-2017-eng.txt',
  'landuse-e.txt',
  'preparedbc_flood_information_for_homeowners_and_home_buyers_2018.txt',
  'protect-your-home-from-basement-flooding.txt',
  'Protect_Your_Home_From_Flooding_Brochure.txt',
  'sea_dike_guidelines.txt',
  'slr-primer.txt',
  'Synthesis_Eng.txt',
  'Urban_Forests_Guide.txt',
  'Vancouver-Climate-Change-Adaptation-Strategy-2012-11-07.txt',
  'WCEL_climate_change_FINAL.txt',
  'Report_on_Effects_of_a_Changing_Climate_to_the_US_Department_of_Defense.txt',
  'Vulnerability_Guidebook_June2_EN.txt',
  'Windsor_Climate_Change_Adaptation_Plan.txt'
  ]

test_path = '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/ClimateChangeDocs_pdfminer'

####Extract 1500 random sentences from the corpus that are not in the Actions list

In [0]:
import os
import re
from random import random, seed

# manually construct a list of eligible files to look at
files = [
  'Guide-Building-Sustainable-and-Resilient-Communities-with-Asset-Management-EN.txt',
  'ccp_impactonpeople.txt',
  'health_facilit-instal_sante-eng.txt',
  'climate_data_discussion_primer.txt',
  'FloodRecovery-e.txt',
  'municipal-climate-change-action-plan-guidebook-en.txt',
  'public_guideline__principles_of_climate_adaptation_and_mitigation_for_engineers.txt',
  'Spring_Flood_Fact_Sheet.txt',
  'Ahead-of-the-Storm-1.txt',
  'builders_guide_2010_final.txt',
  'ClimatRisk-E-ACCESSIBLE.txt',
  '81363.txt',
  'adapt_bulletin-adapt1-eng.txt',
  'ANINTROENGLISH_EC.txt',
  'A_Residential_Guide_to_Flood_Prevention_and_Recovery.txt',
  'Floodproofing.txt',
  'protect-your-home-from-severe-wind-pdf.txt',
  'protect-your-home-from-snow-ice-storms.txt',
  'protect-your-home-from-wildfire.txt',
  'A_Practitioners_Guide_to_ClimateChange_Adaptation_in_Ontarios_Ecosystems_Ver_1_2011.txt',
  'Bruce_2006_AdaptingtoClimateChange_ARisk-basedGuideforONMunicipalities.txt',
  'Adapting_to_Climate_Change_in_Coastal_Communities_In_Canada_White_Paper.txt',
  'Climate_Change_Adaptation_-_A_Priorities_Plan_for_Canada_2012.txt',
  '104890-2017_Climate_Change_Strategy.txt'
  ]

# define variables
non_actions = []    
counter = 0
seed(42)

in_path = '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/ClimateChangeDocs_pdfminer'
out_path = '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/'
outfile = open(out_path+"non-actions-raw.txt", 'w', encoding='utf8')

# read each file in the list
for file in files:
    path_to_txt = os.path.join(in_path, file)
    with open(path_to_txt, encoding='utf8') as f:
        raw_text=f.read()

    text=raw_text
    #text = re.sub(r"([a-zA-Z0-9\s])\s*\n([a-zA-Z0-9\s])", r"\1 \2", text) # remove \n in the middle of a paragraph
    #text = re.sub(r"\\x[0-9A-F]+", " ", text)
    #text = re.sub(r"[^a-zA-Z0-9 \n\.\?\t\r\(\)\{\}\[\]\+\*\$\!\@\&:;\"'=-_%#\^,\\\/]", " ", text) # remove unusual characters

# apply sentence tokenization    
    doc = nlp(text)

    for sent in doc.sents:
#capture random sentences and write them out
        if random() < 0.05:
            if not (sent.text in actions):
                outfile.write(sent.text+'\n')
                non_actions.append(sent.text)
                counter += 1

#stop when 1500 sentences written
    if counter > 1500:
        break

outfile.close()

In [0]:
# raw list of non-actions
non_actions

['An introduction for \n \n',
 '•  City of Vancou',
 'ON\n\n',
 'ON\n\n',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 'A sustainable community is one that meets the needs \n \nof the present without compromising the needs of \n \nfuture generations (source: Environment and Climate \n \nChange Canada).',
 'costs, risks and services \n\n',
 'Integrating natural and built environments \n\n \n\n',
 'Asset management (AM) is an integrated approach, \n \ninvolving all municipal departments, to choosing \n \nand managing existing and new assets.',
 'PROSPERITY \n\n',
 'ti es be built into asset \nmanagement planning? \n\n',
 'Compact, mixed-use de elopment is \ngenerally less costly to ser ice',
 'Re y ling or repurposing assets: As assets come to \n \n',
 'Of course, these considerations are not only  \nimportant at the maintenance stage; climate risks need to  \nbe considered at all stages of the life cycle of assets as well  \nas when decisions are made about building something new. \n\n',
 'CLIM

Manually edit the extracted random sentences to weed out any that are not well-formed sentences or that could be considered Actions even though not labelled as such. This provided 388 samples of non-action sentences.

## Define a function to represent the high-level syntax of a sentence

It should extract the top syntactic levels of the parse tree, excluding the leaves (tokens). 

The syntactic abstraction of the sentence is what will be used for classification.

In [0]:
def parse(text):
# create a spaCy document instance from the text 
    doc = nlp(text)
# find the root of the parse tree on which all other tokens depend
    root = [token for token in doc if token.head == token][0]

# starting from the root, find the syntactic tags of the first two levels of the parse tree.
# using tags instead of pos for greater differentiation of verb types.
    parsed = ['ROOT_self_' + root.tag_]
    for s in root.lefts:
        parsed.extend(['LEFT_' + s.dep_ +'_' + s.tag_])

    for s in root.rights:
        parsed.extend(['RIGHT_' + s.dep_ + '_' + s.tag_])
        
    return parsed

In [0]:
# demonstrate the function's output
parse('I want to be a clone.')

['ROOT_self_VBP', 'LEFT_nsubj_PRP', 'RIGHT_xcomp_VB', 'RIGHT_punct_.']

##Parse and pickle the Actions
 


In [0]:
import csv

#complete path of csv file
csv_path = '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/ClimateChangeDocs-master/Actions/Climate Change Docs - Actions Information Architecture.csv'
outpath = '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/Climate Change Docs - Actions.txt'

# define accumulator 
actions = [] 
parsed_actions = []                                                                   

# open the csv file
with open(csv_path, 'r', encoding="utf8", errors='ignore') as f, open(outpath, 'w') as outfile:          

#   define a reader (generator) for the file                                                                            
    reader = csv.reader(f, delimiter='|')

#   first row is titles
    titles=next(reader)

#   read all the actions  
    for row in reader: 
        if 'citizen' in row[10]:
#   save the original sentences in the actions list         
            actions.append(row[0])
#   parse the syntactic abstraction and pickle it
            parsed = parse(row[0])
            outfile.writelines(' '.join(parsed))
            outfile.write('\n')
#   save the parse for further processing
            parsed_actions.append(parsed)

outfile.close() 

In [0]:
parsed_actions

[['ROOT_self_VB',
  'LEFT_aux_VB',
  'LEFT_nsubj_NN',
  'RIGHT_dobj_NN',
  'RIGHT_advcl_VB',
  'RIGHT_punct_.'],
 ['ROOT_self_VB',
  'LEFT_neg_RB',
  'RIGHT_dobj_NN',
  'RIGHT_prep_IN',
  'RIGHT_advcl_VB',
  'RIGHT_punct_.'],
 ['ROOT_self_VB', 'RIGHT_dobj_NN', 'RIGHT_advcl_VB', 'RIGHT_punct_.'],
 ['ROOT_self_VB',
  'LEFT_nsubj_NNS',
  'LEFT_punct_,',
  'LEFT_advcl_VBN',
  'LEFT_punct_,',
  'LEFT_aux_TO',
  'RIGHT_dobj_NNS',
  'RIGHT_prep_IN',
  'RIGHT_punct_.'],
 ['ROOT_self_VB',
  'RIGHT_dobj_NN',
  'RIGHT_punct_,',
  'RIGHT_advcl_VBG',
  'RIGHT_punct_.'],
 ['ROOT_self_VB', 'RIGHT_dobj_NNS', 'RIGHT_advcl_VB', 'RIGHT_punct_.'],
 ['ROOT_self_VB',
  'LEFT_advcl_JJ',
  'LEFT_punct_,',
  'RIGHT_dobj_NNS',
  'RIGHT_cc_CC',
  'RIGHT_conj_VB',
  'RIGHT_punct_.'],
 ['ROOT_self_VB',
  'RIGHT_prep_IN',
  'RIGHT_cc_CC',
  'RIGHT_conj_VB',
  'RIGHT_punct_.'],
 ['ROOT_self_VB',
  'LEFT_csubj_VB',
  'LEFT_aux_VBP',
  'LEFT_neg_RB',
  'RIGHT_dobj_NNS',
  'RIGHT_punct_.'],
 ['ROOT_self_NNS',
  'LEFT_a

## Parse and pickle the Non-Actions


In [0]:
import csv

#complete path of csv file
inpath = '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/non-actions-edited.txt'
outpath = '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/Climate Change Docs - Non-Actions.txt'

# define accumulator 
non_actions = [] 
parsed_non_actions = []                                                                   

# open the text file
with open(inpath, 'r', encoding="utf8", errors='ignore') as f, open(outpath, 'w') as outfile:          
#   read the non-actions  
    for row in f.readlines(): 
#   save the original sentences in the actions list         
        non_actions.append(row)
#   parse the syntactic abstraction and pickle it
        parsed = parse(row)
        outfile.writelines(' '.join(parsed))
        outfile.write('\n')
#   save the parse for further processing
        parsed_non_actions.append(parsed)
 

In [0]:
parsed_non_actions

[['ROOT_self_VBZ', 'LEFT_nsubj_NN', 'RIGHT_attr_NN'],
 ['ROOT_self_VBZ',
  'LEFT_nsubj_NN',
  'RIGHT_attr_NN',
  'RIGHT_punct_,',
  'RIGHT_advcl_VBG',
  'RIGHT_punct_.'],
 ['ROOT_self_VBP',
  'LEFT_ccomp_VBP',
  'LEFT_punct_,',
  'LEFT_nsubj_NNS',
  'RIGHT_xcomp_VBN',
  'RIGHT_punct_.'],
 ['ROOT_self_VBZ', 'LEFT_nsubj_NN', 'RIGHT_prep_IN', 'RIGHT_punct_.'],
 ['ROOT_self_VBN',
  'LEFT_nsubjpass_PRP',
  'LEFT_auxpass_VBP',
  'RIGHT_xcomp_VB',
  'RIGHT_punct_.'],
 ['ROOT_self_VBZ',
  'LEFT_nsubj_PRP',
  'RIGHT_acomp_JJ',
  'RIGHT_xcomp_VB',
  'RIGHT_punct_.'],
 ['ROOT_self_VB',
  'LEFT_advmod_RB',
  'LEFT_punct_,',
  'LEFT_nsubj_NN',
  'LEFT_aux_MD',
  'RIGHT_prep_IN',
  'RIGHT_punct_,',
  'RIGHT_cc_CC',
  'RIGHT_conj_VB'],
 ['ROOT_self_VB',
  'LEFT_advmod_RB',
  'LEFT_nsubj_NNS',
  'LEFT_aux_MD',
  'RIGHT_dobj_NN',
  'RIGHT_advmod_RB',
  'RIGHT_prep_IN',
  'RIGHT_punct_.'],
 ['ROOT_self_VBZ',
  'LEFT_nsubj_PRP',
  'RIGHT_neg_RB',
  'RIGHT_acomp_JJ',
  'RIGHT_advcl_VB',
  'RIGHT_punct_.']

## Inspect the frequency counts

If there is an obvious rule for identifying actions, hard-code it as a rule.

Otherwise train a classification model on the pickled actions and non-actions.

In [0]:
from nltk import FreqDist

#  count the frequency of the syntactic patterns
action_freqs = FreqDist([' '.join(sent) for sent in parsed_actions])  
non_action_freqs = FreqDist([' '.join(sent) for sent in parsed_non_actions])                    

In [0]:
action_freqs.most_common(20)

[('ROOT_self_VB RIGHT_dobj_NN RIGHT_punct_.', 47),
 ('ROOT_self_VB RIGHT_dobj_NNS RIGHT_punct_.', 28),
 ('ROOT_self_VB RIGHT_dobj_NN RIGHT_prep_IN RIGHT_punct_.', 18),
 ('ROOT_self_VB RIGHT_acomp_JJ RIGHT_punct_.', 17),
 ('ROOT_self_VB RIGHT_prep_IN RIGHT_punct_.', 17),
 ('ROOT_self_VB RIGHT_dobj_NN RIGHT_cc_CC RIGHT_conj_VB RIGHT_punct_.', 11),
 ('ROOT_self_VB RIGHT_dobj_NN RIGHT_advcl_VB RIGHT_punct_.', 9),
 ('ROOT_self_VB RIGHT_cc_CC RIGHT_conj_VB RIGHT_punct_.', 9),
 ('ROOT_self_VB RIGHT_xcomp_VBG RIGHT_punct_.', 8),
 ('ROOT_self_VB RIGHT_dobj_NNS RIGHT_prep_IN RIGHT_punct_.', 7),
 ('ROOT_self_VB RIGHT_dobj_NN', 6),
 ('ROOT_self_VB RIGHT_ccomp_VBN RIGHT_punct_.', 6),
 ('ROOT_self_VB LEFT_nsubj_NN LEFT_aux_MD RIGHT_xcomp_VB RIGHT_punct_.', 6),
 ('ROOT_self_VB LEFT_nsubj_NNS LEFT_aux_MD RIGHT_dobj_NN RIGHT_punct_.', 6),
 ('ROOT_self_VBN LEFT_nsubjpass_NNS LEFT_aux_MD LEFT_auxpass_VB RIGHT_prep_IN RIGHT_punct_.',
  6),
 ('ROOT_self_VB RIGHT_dobj_NNS RIGHT_advcl_VB RIGHT_punct_.', 5),


In [0]:
non_action_freqs.most_common(20)

[('ROOT_self_VBP LEFT_expl_EX RIGHT_advmod_RB RIGHT_attr_NNS RIGHT_punct_.',
  4),
 ('ROOT_self_VBN LEFT_nsubjpass_NN LEFT_auxpass_VBZ RIGHT_xcomp_VB RIGHT_punct_.',
  4),
 ('ROOT_self_VBP LEFT_nsubj_NNS RIGHT_dobj_NN RIGHT_punct_.', 3),
 ('ROOT_self_VBZ LEFT_nsubj_NN RIGHT_attr_NN RIGHT_punct_.', 3),
 ('ROOT_self_VBP LEFT_nsubj_NNS RIGHT_dobj_NNS RIGHT_punct_.', 3),
 ('ROOT_self_NNS RIGHT_prep_IN RIGHT_punct_.', 3),
 ('ROOT_self_VBP LEFT_nsubj_NNS LEFT_advmod_RB RIGHT_prep_IN RIGHT_punct_.',
  3),
 ('ROOT_self_VBZ LEFT_nsubj_NN RIGHT_prep_IN RIGHT_punct_.', 2),
 ('ROOT_self_VBZ LEFT_nsubj_PRP RIGHT_acomp_JJ RIGHT_xcomp_VB RIGHT_punct_.',
  2),
 ('ROOT_self_VB LEFT_nsubj_NN LEFT_aux_MD RIGHT_dobj_NN RIGHT_punct_.', 2),
 ('ROOT_self_VB LEFT_nsubj_NN LEFT_aux_MD RIGHT_dobj_NNS RIGHT_punct_.', 2),
 ('ROOT_self_VB LEFT_nsubj_NNS LEFT_aux_MD RIGHT_ccomp_VB RIGHT_punct_.', 2),
 ('ROOT_self_VBZ LEFT_nsubj_NNP RIGHT_dobj_NN RIGHT_punct_.', 2),
 ('ROOT_self_VBP LEFT_nsubj_NNS RIGHT_attr_NN RIGH

In [0]:
action_freqs['ROOT_self_VB RIGHT_dobj_NN RIGHT_punct_.'] 

47

In [0]:
non_action_freqs['ROOT_self_VB RIGHT_dobj_NN RIGHT_punct_.'] 

0

In [0]:
action_freqs['ROOT_self_VB RIGHT_dobj_NNS RIGHT_punct_.'] 

28

In [0]:
non_action_freqs['ROOT_self_VB RIGHT_dobj_NNS RIGHT_punct_.'] 

0

In [0]:
non_action_freqs['ROOT_self_VBP LEFT_expl_EX RIGHT_advmod_RB RIGHT_attr_NNS RIGHT_punct_.']

4

In [0]:
action_freqs['ROOT_self_VBP LEFT_expl_EX RIGHT_advmod_RB RIGHT_attr_NNS RIGHT_punct_.']

0

In [0]:
non_action_freqs['ROOT_self_VBN LEFT_nsubjpass_NN LEFT_auxpass_VBZ RIGHT_xcomp_VB RIGHT_punct_.']

4

In [0]:
action_freqs['ROOT_self_VBN LEFT_nsubjpass_NN LEFT_auxpass_VBZ RIGHT_xcomp_VB RIGHT_punct_.']

0

## Discussion

There is definitely a difference between the grammatical structures that are common for actions and those that are common for non-actions.

Also, the most common patterns for actions are not found at all for non-actions, and vice versa.

The patterns themselves are too complex to be easily hard-coded by a human, but the fact that the patterns exist is obvious. 

Therefore is seems promising to apply a classification algorithm on the parses.

It may be necessary to balance the size of the datasets, as the actions are approximately twice as many as the non-actions.